In [19]:
!pip -q install transformers sentence_transformers
!pip -q install huggingface_hub langchain
!pip -q install openai 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.3 MB/s eta 0:00:00


In [4]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''
os.environ['OPENAI_API_KEY']=''

## Tasks & its models

1) Question Answering:
deepset/roberta-base-squad2

2) Summarization:sshleifer/distilbart-cnn-12-6

3) Text Classification:distilbert-base-uncased-finetuned-sst-2-english

4) Name Entity recognition:dslim/bert-base-NER

In [5]:
## Using langchain HuggingFace Hub for text-generation integrations
### Model is gpt2
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from transformers import pipeline

In [13]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool
from langchain.llms import OpenAI     

In [7]:
#question answering
model_repo = "deepset/roberta-base-squad2"

qa_model = pipeline("question-answering",
                    model=model_repo)

#Summarisation
model_repo = 'sshleifer/distilbart-cnn-12-6'

summary_pipe = pipeline('summarization',
                        model=model_repo)

#Text Classification

text_classifier = pipeline("text-classification", 
                      model = "roberta-large-mnli")

#NER pipeline

ner_classifier = pipeline("ner",
                      model="dslim/bert-base-NER")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Image Classification :google/vit-base-patch16-224

Image-to-Text:ydshieh/vit-gpt2-coco-en

Object Detection:facebook/detr-resnet-50

In [ ]:
!pip install timm

In [ ]:
cl_model = pipeline("image-classification")
itot = pipeline('image-to-text')
objdet =pipeline("object-detection")

In [11]:
from langchain.agents import tool

### This cell contains all the text related tasks as tools

@tool("question-answering",return_direct=True)
def qa_query(sentence: str) -> str:
    """Replies with the answer for the question."""
    reply = qa_model(sentence)    
    return reply[0]['label']


@tool("summarization",return_direct=True)
def summary_query(sentence: str) -> str:
    """Summarizes the text given to it."""
    reply = summary_pipe(sentence)    
    return reply[0]['label']

@tool("text-classification",return_direct=True)
def txtclass_query(sentence: str) -> str:
    """Useful when ever we need to classify the text available."""
    reply = text_classifier(sentence)    
    return reply[0]['label']

@tool("ner-classification",return_direct=True)
def ner_query(sentence: str) -> str:
    """Useful when ever we need to find entities in the text."""
    reply = ner_classifier(sentence)    
    return reply[0]['label']

In [12]:
### This cell contains all the text related tasks as tools

@tool("image-classification",return_direct=True)
def img_classify(sentence: str) -> str:
    """Useful whenever need to classify the image provided."""
    reply = cl_model(sentence)    
    return reply[0]['label']

@tool("image-text",return_direct=True)
def img_text(sentence: str) -> str:
    """Useful when the image needs to be converted to text."""
    reply = itot(sentence)    
    return reply[0]['label']

@tool("detect_object",return_direct=True)
def detect_obj(sentence: str) -> str:
    """Useful whenever need to detect objects in image provided."""
    reply = cl_model(sentence)    
    return reply[0]['label']

In [14]:
tools_chain = [
    Tool(
        name = "question-answering",
        func=qa_query.run,
        description="Useful when need to answer a question based on the context"
    ),
    Tool(
        name = "summarization",
         func=summary_query.run,
         description="useful when need to Summarizes the text given to it"
    ),
    Tool(
        name = "text-classification",
         func=txtclass_query.run,
         description="Useful when ever we need to classify the text available."
    ),
    Tool(
        name = "ner-classification",
         func=ner_query.run,
         description="Useful when ever we need to get entities from text available."
    ),
    Tool(
        name = "detect_object",
         func=detect_obj.run,
         description="Useful when ever we need to detect objects in the given image."
    ),
    Tool(
        name = "image_text",
         func=img_text.run,
         description="Useful when ever we need to convert the given image to text."
    ),
    Tool(
        name = "image-classification",
         func=img_classify.run,
         description="Useful when ever we need to classify the image provided."
    )
]
     

In [20]:
support_llm = OpenAI(temperature=0)

In [21]:
hugginggpt_clone = initialize_agent(tools_chain,support_llm,
                            agent='zero-shot-react-description',
                            verbose=True)

In [24]:
hugginggpt_clone.save_agent("hugginggpt_clone.json")

In [27]:
text_prompt = """
Between 1980 and 2021, AI research experienced significant growth and development. In the 1980s, expert systems and knowledge-based systems were popular approaches to AI. In the 1990s, machine learning techniques such as neural networks and decision trees gained popularity, leading to advancements in speech recognition, natural language processing, and computer vision. In the 2000s, deep learning and big data revolutionized the field, with breakthroughs in image recognition, language translation, and game playing. In recent years, AI research has focused on ethical concerns, explainability, and transparency, as well as expanding AI applications in fields such as healthcare, finance, and autonomous vehicles.
From the above history of AI, tell me Who was the pioneer in AI research? 
Summarize the AI history into 50 words.
Get the entities in the AI history.
"""

In [28]:
hugginggpt_clone.run(text_prompt)



> Entering new AgentExecutor chain...


Your max_length is set to 142, but you input_length is only 131. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)


 I need to summarize the AI history and get the entities in the text.
Action: summarization
Action Input: Between 1980 and 2021, AI research experienced significant growth and development. In the 1980s, expert systems and knowledge-based systems were popular approaches to AI. In the 1990s, machine learning techniques such as neural networks and decision trees gained popularity, leading to advancements in speech recognition, natural language processing, and computer vision. In the 2000s, deep learning and big data revolutionized the field, with breakthroughs in image recognition, language translation, and game playing. In recent years, AI research has focused on ethical concerns, explainability, and transparency, as well as expanding AI applications in fields such as healthcare, finance, and autonomous vehicles.

KeyError: ignored